PySpark, его зависимости и датасет были установлены локально, поэтому здесь будет только код.

In [15]:
import os
import sys
from pyspark.sql.functions import desc, asc

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

In [16]:
df = spark.read.csv('T_ONTIME_REPORTING.csv', header=True, inferSchema=True)
df = df.union(spark.read.csv('T_ONTIME_REPORTING 2.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('T_ONTIME_REPORTING 3.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('T_ONTIME_REPORTING 4.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('T_ONTIME_REPORTING 5.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('T_ONTIME_REPORTING 6.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('T_ONTIME_REPORTING 7.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('T_ONTIME_REPORTING 8.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('T_ONTIME_REPORTING 9.csv', header=True, inferSchema=True))
df = df.union(spark.read.csv('T_ONTIME_REPORTING 10.csv', header=True, inferSchema=True))

df.show()

+----+--------------------+-----------------+-----------------+------+--------------------+---------------+----+--------------------+---------+---------+---------+-------------------+--------+-------------+-------------+---------+--------------+-------------------+
|YEAR|             FL_DATE|OP_UNIQUE_CARRIER|ORIGIN_AIRPORT_ID|ORIGIN|    ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST|      DEST_CITY_NAME|DEP_DELAY|ARR_DELAY|CANCELLED|ACTUAL_ELAPSED_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+----+--------------------+-----------------+-----------------+------+--------------------+---------------+----+--------------------+---------+---------+---------+-------------------+--------+-------------+-------------+---------+--------------+-------------------+
|2024|1/1/2024 12:00:00 AM|               9E|            10135|   ABE|Allentown/Bethleh...|          10397| ATL|         Atlanta, GA|     -2.0|    -35.0|      0.0|              119.0|   692.0|         N

Задание 1:

    1. Сколько различных перевозчиков представлено в датасете?
    2. Какие 5 перевозчиков сделали наибольшее число перелетов?


In [17]:
print(df.select("OP_UNIQUE_CARRIER").distinct().count())
df.groupBy('OP_UNIQUE_CARRIER').count().orderBy(desc("count")).show()

15


+-----------------+-------+
|OP_UNIQUE_CARRIER|  count|
+-----------------+-------+
|               WN|1179623|
|               DL| 811280|
|               AA| 784153|
|               UA| 602736|
|               OO| 556873|
|               YX| 248120|
|               B6| 229556|
|               NK| 217373|
|               AS| 201987|
|               MQ| 188775|
|               9E| 166489|
|               OH| 161577|
|               F9| 142600|
|               G4|  98120|
|               HA|  67208|
+-----------------+-------+



Всего авиакомпаний 15. Больше всего рейсов выполнили Southwest, Delta, American, United и SkyWest (авиакомпания, которой отданы "на аутсорс" региональные рейсы Delta, American, United и Alaska)

Задание 2. Найдите 10 самых частых направлений, если:

    1. Направление – это пара «аэропорт вылета-аэропорт прилета»
    2. Направление – это пара «город вылета-город прилета»
    3. Направление – это город прилета.


In [18]:
df.groupBy(['ORIGIN', 'DEST']).count().orderBy(desc("count")).show()

df.groupBy(['ORIGIN_CITY_NAME','DEST_CITY_NAME']).count().orderBy(desc("count")).show()

df.groupBy('DEST_CITY_NAME').count().orderBy(desc("count")).show()

+------+----+-----+
|ORIGIN|DEST|count|
+------+----+-----+
|   OGG| HNL|10504|
|   HNL| OGG|10487|
|   LAX| SFO| 9631|
|   SFO| LAX| 9631|
|   LAX| LAS| 8739|
|   LAS| LAX| 8715|
|   ORD| LGA| 8432|
|   LGA| ORD| 8432|
|   DCA| BOS| 8306|
|   BOS| DCA| 8305|
|   JFK| LAX| 8259|
|   LAX| JFK| 8255|
|   DEN| LAS| 7706|
|   LAS| DEN| 7685|
|   PHX| DEN| 7681|
|   DEN| PHX| 7667|
|   ATL| MCO| 7478|
|   MCO| ATL| 7478|
|   LIH| HNL| 7147|
|   HNL| LIH| 7147|
+------+----+-----+
only showing top 20 rows



+-----------------+-----------------+-----+
| ORIGIN_CITY_NAME|   DEST_CITY_NAME|count|
+-----------------+-----------------+-----+
|      Chicago, IL|     New York, NY|12856|
|     New York, NY|      Chicago, IL|12855|
|       Boston, MA|     New York, NY|12337|
|     New York, NY|       Boston, MA|12318|
|      Kahului, HI|     Honolulu, HI|10504|
|     Honolulu, HI|      Kahului, HI|10487|
|        Miami, FL|     New York, NY| 9711|
|     New York, NY|        Miami, FL| 9709|
|San Francisco, CA|  Los Angeles, CA| 9631|
|  Los Angeles, CA|San Francisco, CA| 9631|
|      Atlanta, GA|     New York, NY| 9584|
|     New York, NY|      Atlanta, GA| 9582|
|   Washington, DC|       Boston, MA| 9484|
|       Boston, MA|   Washington, DC| 9481|
|   Washington, DC|      Chicago, IL| 9126|
|      Chicago, IL|   Washington, DC| 9114|
|      Orlando, FL|     New York, NY| 8776|
|     New York, NY|      Orlando, FL| 8774|
|  Los Angeles, CA|    Las Vegas, NV| 8739|
|    Las Vegas, NV|  Los Angeles

+--------------------+------+
|      DEST_CITY_NAME| count|
+--------------------+------+
|         Chicago, IL|282723|
|         Atlanta, GA|277089|
|        New York, NY|247223|
|          Denver, CO|237001|
|Dallas/Fort Worth...|234809|
|       Charlotte, NC|161726|
|     Los Angeles, CA|160652|
|      Washington, DC|157883|
|       Las Vegas, NV|156322|
|         Phoenix, AZ|150906|
|         Houston, TX|137616|
|         Orlando, FL|137357|
|         Seattle, WA|133451|
|          Boston, MA|115888|
|          Newark, NJ|115087|
|   San Francisco, CA|111433|
|         Detroit, MI|103507|
|     Minneapolis, MN|100640|
|  Salt Lake City, UT| 90848|
|           Miami, FL| 87157|
+--------------------+------+
only showing top 20 rows



По аэропортам рейтинг такой:

1. Кахулуи - Гонолулу. Я не ожидал увидеть на первом месте эту пару городов на Гавайях, но это действительно одна из самых активных воздушных линий в США (по состоянию на 8 ноября 2024 года по данным Flightradar24 сейчас между ними 520 рейсов в неделю), хотя по пассажиропотоку она не входит в десятку самых популярных. Причины попадания этого маршрута сразу на первое место, похоже, две: во-первых, значительная часть рейсов на нём выполняют региональные авиалайнеры Boeing 717 на 123-128 мест, что значительно меньше, чем у более популярных на большинстве маршрутов Boeing 737 и Airbus A320, а тем более широкофюзеляжных самолётов. Во-вторых, между этими островными аэропортами интенсивное грузовое сообщение.

2. Сан-Франциско - Лос-Анджелес. Эта пара мегаполисов уже более ожидаемый лидер. По данным Flightradar24 между ними 506 рейсов в неделю.

3. Лос-Анджелес - Лас-Вегас (470 рейсов в неделю)

4. Нью-Йорк, аэропорт LaGuardia - Чикаго, аэропорт O'Hare (441).

5. Вашингтон, аэропорт имени Рейгана - Бостон (387).

6. Нью-Йорк, аэропорт имени Кеннеди - Лос-Анджелес (402).

7. Денвер - Лас-Вегас (305).

8. Денвер - Феникс (379).

9. Атланта - Орландо (333).

10. Лихуэ - Гонолулу (383).

Рейтинг по городам несколько отличается, в первую очередь тем, что выше оказались многие рейсы из Нью-Йорка. Это понятно: в нём два крупных аэропорта: имени Кеннеди (JFK) и LaGuardia (LGA). Кстати, к Нью-Йоркскому авиационному узлу относят и аэропорт Ньюарка (EWR), но в этом датасете ему поставили отдельный город.


Задание 3.
    
    1. Узнайте, сколько было отмен рейсов в каждом месяце. Отсортируйте ответ по числу отмен.
    2. Для каждого месяца определите причину задержки, из-за которой было отложено больше всего рейсов.


In [19]:
from pyspark.sql.functions import split, when, col

col_split = split(df["FL_DATE"], '/')
df = df.withColumn("MONTH", col_split.getItem(0))
print("CANCELLED")
df.filter(col("CANCELLED") == 1).groupBy("MONTH").count().orderBy(desc("count")).show()
for reason in ["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]:
    print(reason)
    df.filter(col(reason) > 0 ).groupBy("MONTH").count().show()

CANCELLED


+-----+-----+
|MONTH|count|
+-----+-----+
|    1|30684|
|    7|14606|
|    6|12219|
|    4| 9589|
|    8| 9172|
|    2| 9019|
|    3| 7406|
|    5| 3310|
|   11|  819|
+-----+-----+

CARRIER_DELAY


+-----+------+
|MONTH| count|
+-----+------+
|    1|129636|
|    2| 52610|
|    3| 74965|
|    4| 71710|
|    5| 62261|
|    6| 87780|
|    7| 96439|
|    8| 73114|
|   11| 44524|
+-----+------+

WEATHER_DELAY


+-----+-----+
|MONTH|count|
+-----+-----+
|    1|18789|
|    2| 4823|
|    3| 6096|
|    4| 6417|
|    5| 4548|
|    6|10342|
|    7|11844|
|    8| 7487|
|   11| 2352|
+-----+-----+

NAS_DELAY


+-----+------+
|MONTH| count|
+-----+------+
|    1|126034|
|    2| 44984|
|    3| 69852|
|    4| 61757|
|    5| 46664|
|    6| 72627|
|    7| 76597|
|    8| 56251|
|   11| 37858|
+-----+------+

SECURITY_DELAY


+-----+-----+
|MONTH|count|
+-----+-----+
|    1| 1490|
|    2|  438|
|    3|  770|
|    4|  735|
|    5|  602|
|    6|  899|
|    7|  793|
|    8|  836|
|   11|  625|
+-----+-----+

LATE_AIRCRAFT_DELAY


+-----+------+
|MONTH| count|
+-----+------+
|    1|117568|
|    2| 43114|
|    3| 68718|
|    4| 66115|
|    5| 53539|
|    6| 83859|
|    7| 94432|
|    8| 69059|
|   11| 35910|
+-----+------+



Во всех месяцах больше всего рейсов было задержано по вине авиакомпаний.

Задание 4. 

Создайте и обучите модель логистической регрессии для предсказания отмены рейса.

In [20]:
from pyspark.sql.types import IntegerType
df=df.withColumn("MONTH", df["MONTH"].cast(IntegerType()))

from pyspark.ml.feature import StringIndexer, OneHotEncoder

carr_indexer = StringIndexer(inputCol="OP_UNIQUE_CARRIER", outputCol="carrier_index")
carr_encoder = OneHotEncoder(inputCol="carrier_index", outputCol="carrier_fact")

origin_indexer = StringIndexer(inputCol="ORIGIN", outputCol="origin_index")
origin_encoder = OneHotEncoder(inputCol="origin_index", outputCol="origin_fact")

dest_indexer = StringIndexer(inputCol="DEST", outputCol="dest_index")
dest_encoder = OneHotEncoder(inputCol="dest_index", outputCol="dest_fact")

df = df.withColumn("label", df.CANCELLED)

from pyspark.ml.feature import VectorAssembler

vec_assembler = VectorAssembler(inputCols=["MONTH", "DISTANCE", "carrier_fact", "dest_fact", "origin_fact"], outputCol="features")

from pyspark.ml import Pipeline

flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, origin_indexer, origin_encoder, vec_assembler])

piped_data = flights_pipe.fit(df).transform(df)

training, test = piped_data.randomSplit([.7, .3])

from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()

import pyspark.ml.evaluation as evals

evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

model = lr.fit(training)
print(model)

test_results = model.transform(test)
print(evaluator.evaluate(test_results))

LogisticRegressionModel: uid=LogisticRegression_56da53744566, numClasses=2, numFeatures=712


0.6662499823805227


    1. Повысится ли качество, если не использовать данные о пункте вылета?
    2. Повысится ли качество, если не использовать данные о пункте прилета?
    3. Повысится ли качество, если не использовать данные о перевозчике?
    4. Повысится ли качество, если не использовать данные о месяце перелета?
    5. Повысится ли качество, если задать направление полета иначе (см. задание 2)?

In [21]:
vec_assembler2 = VectorAssembler(inputCols=["MONTH", "DISTANCE", "carrier_fact", "origin_fact"], outputCol="features")

flights_pipe2 = Pipeline(stages=[origin_indexer, origin_encoder, carr_indexer, carr_encoder, vec_assembler2])

piped_data2 = flights_pipe2.fit(df).transform(df)

training1, test1 = piped_data.randomSplit([.7, .3])

model1 = lr.fit(training1)
print(model1)

test_results1 = model1.transform(test1)
print(evaluator.evaluate(test_results1))

LogisticRegressionModel: uid=LogisticRegression_56da53744566, numClasses=2, numFeatures=712


0.665395675541518


In [22]:
vec_assembler2 = VectorAssembler(inputCols=["MONTH", "DISTANCE", "carrier_fact", "dest_fact"], outputCol="features")

flights_pipe2 = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler2])

piped_data2 = flights_pipe2.fit(df).transform(df)

training2, test2 = piped_data.randomSplit([.7, .3])

model2 = lr.fit(training2)
print(model2)

test_results2 = model2.transform(test2)
print(evaluator.evaluate(test_results2))

LogisticRegressionModel: uid=LogisticRegression_56da53744566, numClasses=2, numFeatures=712


0.6669713832159099


In [23]:
vec_assembler3 = VectorAssembler(inputCols=["MONTH", "DISTANCE", "origin_fact", "dest_fact"], outputCol="features")

flights_pipe3 = Pipeline(stages=[dest_indexer, dest_encoder, origin_indexer, origin_encoder, vec_assembler3])

piped_data3 = flights_pipe3.fit(df).transform(df)

training3, test3 = piped_data.randomSplit([.7, .3])

model3 = lr.fit(training3)
print(model3)

test_results3= model3.transform(test3)
print(evaluator.evaluate(test_results3))

LogisticRegressionModel: uid=LogisticRegression_56da53744566, numClasses=2, numFeatures=712


0.6671393461011693


In [24]:
vec_assembler4 = VectorAssembler(inputCols=[ "DISTANCE", "carrier_fact", "dest_fact", "origin_fact"], outputCol="features")

flights_pipe4 = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, origin_indexer, origin_encoder, vec_assembler4])

piped_data4 = flights_pipe4.fit(df).transform(df)

training4, test4 = piped_data.randomSplit([.7, .3])

model4 = lr.fit(training4)
print(model4)

test_results4 = model4.transform(test4)
print(evaluator.evaluate(test_results4))

LogisticRegressionModel: uid=LogisticRegression_56da53744566, numClasses=2, numFeatures=712


0.6651633599726318


Этого в задании нет, но из любопытства стоит проверить, что будет, если убрать расстояние:

In [25]:
vec_assembler5 = VectorAssembler(inputCols=[ "MONTH", "carrier_fact", "dest_fact", "origin_fact"], outputCol="features")

flights_pipe5 = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, origin_indexer, origin_encoder, vec_assembler5])

piped_data5 = flights_pipe5.fit(df).transform(df)

training5, test5 = piped_data.randomSplit([.7, .3])

model5 = lr.fit(training5)
print(model5)

test_results5 = model5.transform(test5)
print(evaluator.evaluate(test_results5))

LogisticRegressionModel: uid=LogisticRegression_56da53744566, numClasses=2, numFeatures=712


0.6658759298222223


От удаления переменных результат значительно не меняется. 
Теперь попробуем задавать направление через города, а не аэропорты:

In [26]:
origin_city_indexer = StringIndexer(inputCol="ORIGIN_CITY_NAME", outputCol="origin_city_index")
origin_city_encoder = OneHotEncoder(inputCol="origin_city_index", outputCol="origin_city_fact")

dest_city_indexer = StringIndexer(inputCol="DEST_CITY_NAME", outputCol="dest_city_index")
dest_city_encoder = OneHotEncoder(inputCol="dest_city_index", outputCol="dest_city_fact")

vec_assembler6 = VectorAssembler(inputCols=["MONTH", "DISTANCE", "carrier_fact", "dest_city_fact", "origin_city_fact"], outputCol="features")

flights_pipe6 = Pipeline(stages=[dest_city_indexer, dest_city_encoder, carr_indexer, carr_encoder, origin_city_indexer, origin_city_encoder, vec_assembler6])

piped_data6 = flights_pipe6.fit(df).transform(df)

training6, test6 = piped_data6.randomSplit([.7, .3])

model6 = lr.fit(training6)
print(model6)

test_results6 = model6.transform(test6)
print(evaluator.evaluate(test_results6))

LogisticRegressionModel: uid=LogisticRegression_56da53744566, numClasses=2, numFeatures=700


0.6670895973089321


Результат всё ещё на уровне 2 из 3.